In [1]:
import pandas as pd 
import numpy as np 
import os 
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import euclidean_distances

In [12]:
money_puck = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/other/mp_all_skater.csv')
nhl_api = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/all_players.csv')

In [13]:
money_puck.head()

,Unnamed: 0,name,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,player_id,shifts.1,shots_blocked_by_player,assists,position_encoded
0,0,A.J. Greer,206.0,39.0,13.0,7.0,23.0,274.0,5.0,135.0,...,166,86117.0,3.12,3.22,41.0,8478421,2109.0,65.0,18.0,2
1,1,Aaron Downey,0.0,2.0,1.0,0.0,0.0,11.0,0.0,1.0,...,4,1250.0,0.45,0.48,2.0,8465992,34.0,1.0,1.0,3
2,2,Aaron Ekblad,2658.0,621.0,114.0,28.0,70.0,744.0,62.0,2022.0,...,673,921635.0,5.44,5.40,114.0,8477932,18219.0,723.0,229.0,1
3,3,Aaron Gagnon,57.0,2.0,3.0,0.0,3.0,24.0,1.0,35.0,...,38,19110.0,2.00,1.98,1.0,8471451,481.0,14.0,2.0,0
4,4,Aaron Johnson,317.0,62.0,10.0,4.0,7.0,229.0,6.0,191.0,...,146,136534.0,1.98,1.99,24.0,8469534,3018.0,156.0,24.0,1


In [11]:
len(money_puck)

2863

In [4]:
nhl_api.head()

,player_id,gameCount,goals,assists,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,pim
0,8449795,66.0,11.0,17.0,28.0,-3.0,0.0,0.0,1.0,0.0,92.0,0.0,33.0
1,8467468,20.0,1.0,3.0,4.0,-8.0,0.0,1.0,0.0,0.0,14.0,417.0,4.0
2,8470666,57.0,4.0,1.0,5.0,-1.0,0.0,0.0,2.0,0.0,44.0,748.0,100.0
3,8451858,40.0,9.0,8.0,17.0,-9.0,4.0,8.0,0.0,0.0,85.0,0.0,16.0
4,8468001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0


In [18]:
nhl_api_grouped =  nhl_api.groupby(by=['player_id']).sum().reset_index()
nhl_api_grouped

,player_id,gameCount,goals,assists,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,pim
0,8444869,352.0,22.0,108.0,130.0,-36.0,8.0,50.0,0.0,0.0,582.0,0.0,84.0
1,8444882,3024.0,434.0,344.0,778.0,-268.0,48.0,50.0,98.0,0.0,4040.0,0.0,744.0
2,8444890,598.0,0.0,92.0,92.0,-322.0,0.0,46.0,0.0,0.0,414.0,0.0,184.0
3,8444892,256.0,48.0,88.0,136.0,-24.0,32.0,56.0,8.0,0.0,528.0,0.0,128.0
4,8444893,144.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,36.0,0.0,520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6143,8484321,144.0,0.0,0.0,0.0,-8.0,0.0,0.0,0.0,0.0,200.0,2728.0,48.0
6144,8484387,2035.0,592.0,740.0,1332.0,-555.0,222.0,518.0,111.0,111.0,4255.0,39257.0,1406.0
6145,8484801,1530.0,578.0,782.0,1360.0,-680.0,204.0,442.0,136.0,34.0,5100.0,30430.0,680.0
6146,8484911,238.0,0.0,68.0,68.0,-136.0,0.0,34.0,0.0,0.0,238.0,4556.0,0.0


In [19]:
# Check if playerId in money_puck is in nhl_api_grouped
money_puck['playerId_in_nhl_api'] = money_puck['player_id'].isin(nhl_api_grouped['player_id'])

# Display the rows where playerId is in nhl_api_grouped
matches = money_puck[money_puck['playerId_in_nhl_api']]

# Display the matches
print(matches.head())

# Check the number of matches
num_matches = matches.shape[0]
print(f'Number of matches: {num_matches}')

# Check the number of unique playerIds in each DataFrame
num_unique_money_puck = money_puck['player_id'].nunique()
num_unique_nhl_api = nhl_api_grouped['player_id'].nunique()
print(f'Number of unique playerIds in money_puck: {num_unique_money_puck}')
print(f'Number of unique playerIds in nhl_api_grouped: {num_unique_nhl_api}')

   Unnamed: 0           name  d_zone_shift_starts  giveaways  goals  \
0           0     A.J. Greer                206.0       39.0   13.0   
1           1   Aaron Downey                  0.0        2.0    1.0   
2           2   Aaron Ekblad               2658.0      621.0  114.0   
3           3   Aaron Gagnon                 57.0        2.0    3.0   
4           4  Aaron Johnson                317.0       62.0   10.0   

   high_danger_goals  high_danger_shots   hits  low_danger_goals  \
0                7.0               23.0  274.0               5.0   
1                0.0                0.0   11.0               0.0   
2               28.0               70.0  744.0              62.0   
3                0.0                3.0   24.0               1.0   
4                4.0                7.0  229.0               6.0   

   low_danger_shots  ...   icetime  on_ice_corsi_percentage  \
0             135.0  ...   86117.0                     3.12   
1               1.0  ...    1250.0    

In [22]:
# Rename the player_id column in nhl_api_grouped to match player_id in money_puck
nhl_api_grouped = nhl_api_grouped.rename(columns={'player_id': 'player_id'})

# Merge the DataFrames on the player_id column
merged_df = pd.merge(money_puck, nhl_api_grouped, on='player_id', how='inner')

# Display the resulting DataFrame
merged_df

,Unnamed: 0,name,d_zone_shift_starts,giveaways,goals_x,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,assists_y,points_y,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts_y,pim
0,0,A.J. Greer,206.0,39.0,13.0,7.0,23.0,274.0,5.0,135.0,...,90.0,164.0,16.0,0.0,0.0,10.0,0.0,977.0,9961.0,664.0
1,1,Aaron Downey,0.0,2.0,1.0,0.0,0.0,11.0,0.0,1.0,...,137.0,252.0,10.0,0.0,0.0,12.0,0.0,822.0,14038.0,3824.0
2,2,Aaron Ekblad,2658.0,621.0,114.0,28.0,70.0,744.0,62.0,2022.0,...,184.0,208.0,64.0,0.0,72.0,16.0,0.0,752.0,11304.0,344.0
3,3,Aaron Gagnon,57.0,2.0,3.0,0.0,3.0,24.0,1.0,35.0,...,38.0,122.0,-1.0,0.0,0.0,28.0,0.0,479.0,7912.0,56.0
4,4,Aaron Johnson,317.0,62.0,10.0,4.0,7.0,229.0,6.0,191.0,...,36.0,60.0,-36.0,6.0,12.0,0.0,0.0,232.0,5530.0,230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,2858,Zack Stortini,196.0,27.0,10.0,3.0,14.0,286.0,3.0,84.0,...,48.0,48.0,-24.0,0.0,0.0,0.0,0.0,220.0,4148.0,1108.0
2799,2859,Zbynek Michalek,2549.0,207.0,24.0,3.0,9.0,521.0,16.0,890.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,24.0,544.0,0.0
2800,2860,Zdeno Chara,4476.0,796.0,124.0,34.0,82.0,1802.0,69.0,2736.0,...,96.0,112.0,64.0,0.0,0.0,0.0,0.0,648.0,13408.0,680.0
2801,2861,Zemgus Girgensons,2284.0,118.0,89.0,25.0,94.0,1239.0,22.0,870.0,...,232.0,474.0,-233.0,0.0,21.0,32.0,0.0,4282.0,59561.0,1286.0
